<a href="https://colab.research.google.com/github/jessejk484/Jashwanthkalyan_INFO5731_SPRING2023/blob/main/In_class_exercise_04_03282023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

cars_data = []
webpages = 3
for i in range(webpages):
  url = f"https://www.cars.com/shopping/denton-tx/?page={i}"
  data = requests.get(url)
  soup = BeautifulSoup(data.content, "html.parser")
  car_names = soup.findAll("a", attrs={"data-linkname":"vehicle-listing"})
  for name in car_names:
    car_name = name.text.strip().replace("\n","")
    #car_url = name.get("href")
    car_parent = name.parent
    if not car_parent.find("span", attrs={"class": "sds-badge__label"}):
      continue
    if not car_parent.find("div", attrs={"class": "mileage"}):
      continue
    if not car_parent.find("div", attrs={"class": "dealer-name"}):
      continue
    if not car_parent.find("div", attrs={"class": "price-section price-section-vehicle-card"}):
      continue
    if not car_parent.find("div", attrs={"class": "sds-rating"}):
      continue
    category = car_parent.find("span", attrs={"class": "sds-badge__label"}).text.strip().replace("\n","")
    mileage = car_parent.find("div", attrs={"class": "mileage"}).text.strip().replace(",","").replace("mi.","").replace("\n","")
    dealer =  car_parent.find("div", attrs={"class": "dealer-name"}).text.strip().replace("\n","")
    price = car_parent.find("div", attrs={"class": "price-section price-section-vehicle-card"}).text.strip().replace("$","").replace(",","").replace("\n","")
    rating = car_parent.find("div", attrs={"class": "sds-rating"}).text.strip().replace("()","").replace("\n","")
    cars_data.append([car_name, category, mileage, dealer, price, rating])
  time.sleep(4)
  print("Done with page: {}".format(i+1))

cars_data_frame = pd.DataFrame(cars_data, columns=["Car Name", "Category", "Mileage", "Dealer-Name", "Price", "rating"])
cars_data_frame.head()

Done with page: 1
Done with page: 2
Done with page: 3


,Car Name,Category,Mileage,Dealer-Name,Price,rating
0,2015 Lincoln MKS Base,Good Deal,76513,Honda of Denton,16998,4.4(721 reviews)
1,2012 Mercedes-Benz CL-Class CL 550 4MATIC,Good Deal,67788,AutoNation Honda Lewisville,28626,"4.5(1,734 reviews)"
2,1988 Land Rover Defender,Great Deal,171096,Reserve Auto Group,50988,4.0(51 reviews)
3,2019 Honda Odyssey EX,Good Deal,132718,David McDavid Honda of Frisco,20488,"4.1(1,447 reviews)"
4,2020 Volvo XC90 T5 Momentum 7 Passenger,Fair Deal,82797,Crest Volvo Cars,34777,"4.9(1,718 reviews)"


In [2]:
for x in range(len(cars_data_frame)):
  cars_data_frame.loc[x, "rating"] = cars_data_frame.loc[x, "rating"].split("(")[0]

cars_data_frame.head()

,Car Name,Category,Mileage,Dealer-Name,Price,rating
0,2015 Lincoln MKS Base,Good Deal,76513,Honda of Denton,16998,4.4
1,2012 Mercedes-Benz CL-Class CL 550 4MATIC,Good Deal,67788,AutoNation Honda Lewisville,28626,4.5
2,1988 Land Rover Defender,Great Deal,171096,Reserve Auto Group,50988,4.0
3,2019 Honda Odyssey EX,Good Deal,132718,David McDavid Honda of Frisco,20488,4.1
4,2020 Volvo XC90 T5 Momentum 7 Passenger,Fair Deal,82797,Crest Volvo Cars,34777,4.9


In [3]:
for x in range(len(cars_data_frame)):
  if cars_data_frame.loc[x,"rating"]:
    cars_data_frame.loc[x, "rating"] = round(float(cars_data_frame.loc[x, "rating"]),0)
  else:
    cars_data_frame.loc[x, "rating"] = 0

cars_data_frame.head()

,Car Name,Category,Mileage,Dealer-Name,Price,rating
0,2015 Lincoln MKS Base,Good Deal,76513,Honda of Denton,16998,4.0
1,2012 Mercedes-Benz CL-Class CL 550 4MATIC,Good Deal,67788,AutoNation Honda Lewisville,28626,4.0
2,1988 Land Rover Defender,Great Deal,171096,Reserve Auto Group,50988,4.0
3,2019 Honda Odyssey EX,Good Deal,132718,David McDavid Honda of Frisco,20488,4.0
4,2020 Volvo XC90 T5 Momentum 7 Passenger,Fair Deal,82797,Crest Volvo Cars,34777,5.0


In [4]:
cars_data_frame['rating'] = cars_data_frame['rating'].astype(int)
cars_data_frame['Mileage'] = cars_data_frame['Mileage'].astype(int)
cars_data_frame['Price'] = cars_data_frame['Price'].astype(int)
cars_data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Car Name     33 non-null     object
 1   Category     33 non-null     object
 2   Mileage      33 non-null     int64 
 3   Dealer-Name  33 non-null     object
 4   Price        33 non-null     int64 
 5   rating       33 non-null     int64 
dtypes: int64(3), object(3)
memory usage: 1.7+ KB


In [5]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import nltk
nltk.download('stopwords')
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  !java -version
install_java()
!pip install pyLDAvis

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 38.9 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [7]:
!pip install --upgrade gensim
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
import lxml.html as LH
import urllib.request as urllib2
from bs4 import BeautifulSoup
import spacy.cli
spacy.cli.download("en")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [9]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [10]:
# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    

NameError: ignored

In [ ]:

# loop through document list
for i in cars_data_frame:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install stop-words

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32893 sha256=07a48163021c754f59f66248cf6990d36182e267152c823daeb43f8e7f5e64ec
  Stored in directory: c:\users\jaswa\appdata\local\pip\cache\wheels\da\d8\66\395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
Successfully built stop-words
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
# Write your code here





## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
# Write your code here





## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)



